## Unsupervised Learning - Clustering - K-Means


Modify the scratch code of K-means clustering in our lecture:
- Modify so it print out the total within-cluster variation.  Then try to run several k and identify which k is best.
- Since k-means can be slow due to its pairwise computations, let's implement a mini-batch k-means in which the cluster is create using only partial subset of samples.
- Put everything into a class

Mini-Batch will rarely converge, thus it is important to add a max_iteration or some tolerance.  Last, theoretically speaking, Mini-Batch will never perform better in terms of accuracy when compare to K-means, but it is very close to optimal but will almost always beat K-means in terms of time given large dataset and a modest tolerance parameter.

#### st122645

In [109]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.metrics import pairwise_distances_argmin
from time import time


In [110]:
X, y_true = make_blobs(n_samples=1500, centers=4, cluster_std=0.60, random_state=0)

In [113]:
class KM:
    def __init__(self,n_clusters, replacement = True, batch_size = 100, max_iter = 100):
        self.n_clusters = n_clusters
        self.replacement=replacement
        self.batch_size = batch_size
        self.max_iter = max_iter
        
    def fit(self, X):
        m, n = X.shape

        # 1. randomly choose n clusters from X; you can also randomly generate any two points
        rng = np.random.RandomState(94)
        i = rng.permutation(m)[:self.n_clusters]
        self.centers = X[i]

        iteration = 0

        while True:
            # 2. assign lables based on closest center
            # return the index of centers having smallest distance with X
            labels = pairwise_distances_argmin(X, self.centers)

            # 3. find new centers
            new_centers = []
            for i in range(self.n_clusters):
                new_centers.append(X[labels == i].mean(axis=0))

            # convert list to np.array; you can actually combine 3 with np.array in one sentence 
            new_centers = np.array(new_centers)

            # plotting purpose
            # plot every 5th iteration to save space; remove this if, if you want to see each snapshot
            if (iteration % 5 == 0):
                pred = pairwise_distances_argmin(X, new_centers)
#                 plt.figure(figsize=(5, 2))
#                 plt.title(f"Iteration: {iteration}")
#                 plt.scatter(X[:, 0], X[:, 1], c=pred)
#                 plt.scatter(new_centers[:, 0], new_centers[:, 1], s=100, c="black", alpha=0.6)

            # 4 stopping criteria - if centers do not change anymore, we stop!
            if(np.allclose(self.centers, new_centers)):
                break
            else:
                self.centers = new_centers
                iteration+=1

        print(f"Done in {iteration} iterations")
#         return centers

        total_with_variation_score = 0
        labels = pairwise_distances_argmin(X, self.centers) #<---Note I use X here.  Why?
        for i in range(self.n_clusters):
            cluster_mean = X[labels==i].mean(axis=0)
            total_with_variation_score += ((X[labels==i] - cluster_mean)** 2).sum()
            
        print("Total with variation score: ", total_with_variation_score)
        
    def predict(self, X):
        return pairwise_distances_argmin(X, self.centers)


In [117]:
for n_clusters in range(1,9):
    print(f"=====k = {n_clusters}")
    start = time()
    model = KM(n_clusters)
    model.fit(X)
    preds = model.predict(X)
    print(f"Fit and predict time {time() - start}")
   

=====k = 1
Done in 1 iterations
Total with variation score:  14005.126690305286
Fit and predict time 0.003000497817993164
=====k = 2
Done in 10 iterations
Total with variation score:  5805.884294336651
Fit and predict time 0.008002281188964844
=====k = 3
Done in 8 iterations
Total with variation score:  2493.8460360964195
Fit and predict time 0.00599980354309082
=====k = 4
Done in 3 iterations
Total with variation score:  1006.3420400278767
Fit and predict time 0.0030014514923095703
=====k = 5
Done in 12 iterations
Total with variation score:  921.7604906042242
Fit and predict time 0.008001327514648438
=====k = 6
Done in 12 iterations
Total with variation score:  844.5064140178886
Fit and predict time 0.00800180435180664
=====k = 7
Done in 12 iterations
Total with variation score:  764.1291155529301
Fit and predict time 0.009001970291137695
=====k = 8
Done in 13 iterations
Total with variation score:  663.9292594307542
Fit and predict time 0.010002613067626953


In [125]:
# for n_clusters in range(1,9):
#     plt.figure()
#     plt.scatter(X[:, 0], X[:, 1], c=preds, s=50)
#     plt.title("Result")
